In [55]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 784
l1_size = 784
l2_size = 28
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [57]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, l1_size, l2_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, l1_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(l1_size, l2_size)
        self.l3 = nn.Linear(l2_size, num_classes)
        
    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out
    
model = NeuralNet(input_size, l1_size, l2_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_step = len(train_loader)

for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1) % 100 == 0:
            print(f'Epoch: {epoch+1}, Step: {i+1}, Loss: {loss.item()}')

Epoch: 1, Step: 100, Loss: 0.2384181171655655
Epoch: 1, Step: 200, Loss: 0.3043947219848633
Epoch: 1, Step: 300, Loss: 0.32601550221443176
Epoch: 1, Step: 400, Loss: 0.24188081920146942
Epoch: 1, Step: 500, Loss: 0.2499607652425766
Epoch: 1, Step: 600, Loss: 0.230823814868927
Epoch: 2, Step: 100, Loss: 0.1963920146226883
Epoch: 2, Step: 200, Loss: 0.10965041816234589
Epoch: 2, Step: 300, Loss: 0.23000866174697876
Epoch: 2, Step: 400, Loss: 0.09052956849336624
Epoch: 2, Step: 500, Loss: 0.07735075801610947
Epoch: 2, Step: 600, Loss: 0.10026448965072632


In [58]:
with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)
    
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs,1)
        n_correct += (predicted == labels).sum().item()
        
    print(f'Accuracy = {n_correct} / {n_samples} = {n_correct / n_samples}')

Accuracy = 9697 / 10000 = 0.9697
